In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [18]:
import yfinance as yf
import pandas as pd
import numpy as pd
# import pyspark.sql.SQLContext
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as F
from pyspark.sql import Window

spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [28]:
tsla = yf.Ticker("TSLA")
tsla

yfinance.Ticker object <TSLA>

In [29]:
# apple = spark.createDataFrame(aapl)
tsla.info

{'zip': '94304',
 'sector': 'Consumer Cyclical',
 'fullTimeEmployees': 48016,
 'longBusinessSummary': 'Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, Netherlands, Norway, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offers sedans and sport utility vehicles. It also provides electric powertrain components and systems; and services for electric vehicles through its company-owned service locations, and Tesla mobile service technicians, as well as sells used vehicles. This segment markets and sells its products through a network of company-owned stores and galleries, as well as through its own Website. The Energy Generation and Storage segment offers energy storage products, such as rechargeable lithium-ion battery systems for use in homes, industrial, commercial facilities, and utility grids; and desig

In [30]:
hist = tsla.history(period="14d")
hist

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2020-04-22,703.98,734.00,688.71,732.11,14224800,0,0
2020-04-23,727.60,734.00,703.13,705.63,13236700,0,0
2020-04-24,710.81,730.73,698.18,725.15,13237600,0,0
2020-04-27,737.61,799.49,735.00,798.75,20681400,0,0
2020-04-28,795.64,805.00,756.69,769.12,15222000,0,0
2020-04-29,790.17,803.20,783.16,800.51,16216000,0,0
2020-04-30,855.19,869.82,763.50,781.88,28471900,0,0
2020-05-01,755.00,772.77,683.04,701.32,32531800,0,0
2020-05-04,701.00,762.00,698.00,761.19,19237100,0,0


In [31]:
tesla = spark.createDataFrame(hist)

In [32]:
tesla.describe()

DataFrame[summary: string, Open: string, High: string, Low: string, Close: string, Volume: string, Dividends: string, Stock Splits: string]

trying to show the ability to use spark sql below

In [34]:
tesla = spark.sql("SHOW COLUMNS FROM tesla")

AnalysisException: 'java.lang.RuntimeException: java.lang.RuntimeException: Unable to instantiate org.apache.hadoop.hive.ql.metadata.SessionHiveMetaStoreClient;'

In [25]:
sparkdf = sqlContext.createDataFrame(hist.reset_index(drop=False))
sparkdf

DataFrame[Date: timestamp, Open: double, High: double, Low: double, Close: double, Volume: bigint, Dividends: double, Stock Splits: bigint]

In [27]:
closings = []
for row in sparkdf.itertuples():
    for closing in row[2].split('.'):
        if closing != '':
            closing.append((row[1], closing))
new_df = pandas.sparkdf(closings, columns=['DATE', 'CLOSE'])

AttributeError: 'DataFrame' object has no attribute 'itertuples'

In [9]:
sparkdf.show()

+-------------------+------+------+------+------+--------+---------+------------+
|               Date|  Open|  High|   Low| Close|  Volume|Dividends|Stock Splits|
+-------------------+------+------+------+------+--------+---------+------------+
|2020-04-21 00:00:00|275.53| 276.5|264.71|267.65|45247900|      0.0|           0|
|2020-04-22 00:00:00|272.87|277.15|271.47|275.35|29264300|      0.0|           0|
|2020-04-23 00:00:00|275.13|280.99|274.13|274.29|31203600|      0.0|           0|
|2020-04-24 00:00:00|276.45|282.25|276.25|282.21|31627200|      0.0|           0|
|2020-04-27 00:00:00|281.04|283.77|279.19|282.41|29271900|      0.0|           0|
|2020-04-28 00:00:00|284.31|285.06|277.45|277.83|28001200|      0.0|           0|
|2020-04-29 00:00:00|283.96|288.89|283.12|286.95|34320200|      0.0|           0|
|2020-04-30 00:00:00|289.18|293.73|287.57|293.01|45766000|      0.0|           0|
|2020-05-01 00:00:00|285.48|298.19|285.08|288.29|60154200|      0.0|           0|
|2020-05-04 00:0

In [30]:
sparkdf.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: long (nullable = true)
 |-- Dividends: double (nullable = true)
 |-- Stock Splits: long (nullable = true)



In [29]:
sparkdf.count()

14

In [31]:
sparkdf.limit(5).show()

+-------------------+------+------+------+------+--------+---------+------------+
|               Date|  Open|  High|   Low| Close|  Volume|Dividends|Stock Splits|
+-------------------+------+------+------+------+--------+---------+------------+
|2020-04-21 00:00:00|275.53| 276.5|264.71|267.65|45247900|      0.0|           0|
|2020-04-22 00:00:00|272.87|277.15|271.47|275.35|29264300|      0.0|           0|
|2020-04-23 00:00:00|275.13|280.99|274.13|274.29|31203600|      0.0|           0|
|2020-04-24 00:00:00|276.45|282.25|276.25|282.21|31627200|      0.0|           0|
|2020-04-27 00:00:00|281.04|283.77|279.19|282.41|29271900|      0.0|           0|
+-------------------+------+------+------+------+--------+---------+------------+



In [25]:
sparkdf.createOrReplaceTempView("hist")


In [15]:
histdf = spark.sql('SELECT * MINUS(Close) OVER (PARTITION BY Date ORDER BY Close) as Close_prev FROM hist')

# sparkdf = sparkdf.withColumn("Close_prev", F.lag(sparkdf.Close))
# sparkdf = sparkdf.withColumn("Close_diff", F.when(F.isnull(sparkdf.Close - sparkdf.Close_prev), 0)
#                    .otherwise(sparkdf.Close - sparkdf.Close_prev))
spark.sql(query).show()

ParseException: "\nmismatched input 'Close' expecting {'(', 'SELECT', 'FROM', 'VALUES', 'TABLE', 'INSERT', 'MAP', 'REDUCE'}(line 1, pos 15)\n\n== SQL ==\nSELECT * MINUS(Close) OVER (PARTITION BY Date ORDER BY Close) as Close_prev FROM hist\n---------------^^^\n"

Define MYSQL conneciton properties

In [54]:
sparkdf.write.jbdc(
    url='jdbc:mysql://localhost:3306/stocks',
    table='mystock',
    mode='append',
    properties={'user':'root','password':'yourpassword','driver':'com.mysql.cj.jdbc.Driver'}

)

AttributeError: 'DataFrameWriter' object has no attribute 'jbdc'